In [1]:
from torchvision import datasets, transforms

In [16]:
# we'll see if needed
# Define transforms to preprocess the data (you can customize these as needed)
transform = transforms.Compose([
    transforms.Pad((105, 90, 106, 90), fill=0),  # Add Padding:  Our images are 64x33 (left,right,top, bottom)
    #transforms.Resize((244, 244)),
    #transforms.CenterCrop(224),         # Crop the center 224x224 portion of the image
    transforms.ToTensor(),              # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image
])

In [17]:
# Load the dataset using ImageFolder
dataset = datasets.ImageFolder(root="dataset", transform=transform)

In [18]:
dataset[0][0].shape

torch.Size([3, 260, 228])